In [1]:
%load_ext tensorboard

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# read data
df = pd.read_csv("ML_data_unscaled_dummys_encoded.csv")
df = df.drop('Entry_ID',axis=1)
df.head()

,Age,Hours_per_day,BPM,Anxiety,Depression,Insomnia,OCD,While_working_No,While_working_Yes,Instrumentalist_No,...,Rock_Rarely,Rock_Sometimes,Rock_Very frequently,Video game music_Never,Video game music_Rarely,Video game music_Sometimes,Video game music_Very frequently,Music_effects_Improve,Music_effects_No effect,Music_effects_Worsen
0,18.0,4.0,132.0,7.0,7.0,10.0,2.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,61.0,2.5,84.0,9.0,7.0,3.0,3.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,18.0,4.0,107.0,7.0,2.0,5.0,9.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,18.0,5.0,86.0,8.0,8.0,7.0,7.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,18.0,3.0,66.0,4.0,8.0,6.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


## Mental improvement model

In [4]:
# collapse target columns
target_cols = ['Music_effects_Improve','Music_effects_No effect','Music_effects_Worsen']
target_col = df[target_cols].idxmax(1)
target_col.value_counts()

Music_effects_Improve      385
Music_effects_No effect    113
Music_effects_Worsen        13
dtype: int64

In [5]:
# encode target
y = target_col.replace({'Music_effects_Improve':1,'Music_effects_No effect':0,'Music_effects_Worsen':0}).values

In [6]:
# split data into features and target
X = df.drop(columns=target_cols).values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
# # Create a StandardScaler instances
# scaler = StandardScaler()

# # Fit the StandardScaler
# X_scaler = scaler.fit(X_train)

# # Scale the data
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    # activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=32, max_value=512, step=32), activation=hp.Choice('activation',['relu','tanh','sigmoid'])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=32, max_value=512, step=32),
            activation=hp.Choice('activation',['relu','tanh','sigmoid'])))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [21]:
# # Import the kerastuner library
# import kerastuner as kt

# tuner = kt.Hyperband(
#     create_model,
#     objective="val_accuracy",
#     max_epochs=50,
#     hyperband_iterations=2,
#     directory='imp_hyp',
#     overwrite=True,
#     project_name='imp_hyp')

In [20]:
# # Run the kerastuner search for best hyperparameters
# tuner.search(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 00m 04s]
val_accuracy: 0.75

Best val_accuracy So Far: 0.7578125
Total elapsed time: 00h 05m 12s
INFO:tensorflow:Oracle triggered exit


In [11]:
# # Get best model hyperparameters
# best_hyper = tuner.get_best_hyperparameters(1)[0]
# best_hyper.values

{'first_units': 32,
 'activation': 'relu',
 'num_layers': 1,
 'units_0': 64,
 'units_1': 224,
 'units_2': 192,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 3,
 'tuner/round': 0}

In [12]:
# # Evaluate best model against test data
# best_model = tuner.get_best_models(1)[0]
# model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 0.5903 - accuracy: 0.7734 - 147ms/epoch - 37ms/step
Loss: 0.5903429985046387, Accuracy: 0.7734375


Loss: 0.5903429985046387, Accuracy: 0.7734375

In [13]:
# best_model.save("models/trained_imp.h5")

### Trying again without mental health ratings

In [8]:
X2 = df.drop(columns=target_cols)
X2 = X2.drop(columns=['Depression','Anxiety','Insomnia','OCD']).values

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, random_state=42)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X2_scaler = scaler.fit(X2_train)

# Scale the data
X2_train_scaled = X2_scaler.transform(X2_train)
X2_test_scaled = X2_scaler.transform(X2_test)

In [10]:
# Import the kerastuner library
import kerastuner as kt

directory = 'logs/imp_hyp'

tuner2 = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2,
    directory=directory,
    overwrite=True,
    project_name='imp_hyp')

c:\Users\mbtho\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [11]:
#log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f'{directory}/fit', histogram_freq=1)

In [12]:
# Run the kerastuner search for best hyperparameters
tuner2.search(X2_train_scaled,y2_train,epochs=50,validation_data=(X2_test_scaled,y2_test),callbacks=[tensorboard_callback])

Trial 180 Complete [00h 00m 06s]
val_accuracy: 0.75

Best val_accuracy So Far: 0.7734375
Total elapsed time: 00h 05m 49s
INFO:tensorflow:Oracle triggered exit


In [41]:
# Get best model hyperparameters
best_hyper2 = tuner2.get_best_hyperparameters(1)[0]
best_hyper2.values

{'first_units': 32,
 'activation': 'tanh',
 'num_layers': 2,
 'units_0': 160,
 'units_1': 288,
 'units_2': 64,
 'tuner/epochs': 17,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 1,
 'tuner/round': 0}

In [42]:
# Evaluate best model against test data
best_model2 = tuner2.get_best_models(1)[0]
model_loss, model_accuracy = best_model2.evaluate(X2_test_scaled,y2_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 0.5877 - accuracy: 0.7734 - 164ms/epoch - 41ms/step
Loss: 0.5877113342285156, Accuracy: 0.7734375


In [43]:
best_model2.save("models/trained_imp.h5")

In [31]:
%reload_ext tensorboard

In [40]:
%tensorboard --logdir logs/imp_hyp